In [14]:
# Import necessary Python modules
from sklearn.decomposition import PCA
from mne_icalabel import label_components
from mne.preprocessing import ICA
import copy  # This is a Python module that allows you to copy objects without changing the original object
from scipy import signal
import sklearn as sk
import matplotlib.pyplot as plt
import os
import matplotlib
import mne
import numpy as np
from scipy.stats import zscore
mne.viz.set_browser_backend('matplotlib', verbose=None)

# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}

ch_map_fix = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
              # Setting FPz as GND so it matches montage
              'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
              'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
              'Ch19': 'P4', 'Ch20': 'P8', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
              'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
              'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
              'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
              'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
              'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
              'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
              'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye', 'Aux1': 'Rmastoid',}

for subject in subject_list:
    for mode in modes:
        # defining paths for current subject

        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'

        log_file = output_path+'log1.txt'
        bad_channel_file = output_path+'bad_channel.txt'
        bad_channel = open(bad_channel_file, "w")
        log = open(log_file, "w")

        # debug line
        log.write("Reading in .vhdr at " + input_path + 'TCOA_' +
                  subject + '_'+mode+'.vhdr' + "\n \n \n \n")
        
        EEG = None
        # loading in VHDR file
        if '116' in subject:
            EEG = mne.io.read_raw_eeglab(input_path +'116.set', preload = True)
        else:
            continue
            EEG = mne.io.read_raw_brainvision(
                input_path + 'TCOA_' +
                subject + '_'+mode+'.vhdr', preload=True)
        print('Success')
        # except:  # skip if error
        #     log.write("ERROR Reading in .vhdr at " +
        #               input_path + 'TCOA_' +
        #               subject + '_'+mode+'.vhdr' + "\n")
        #     continue

        # drop channels
        print(EEG.ch_names)

        if len(EEG.ch_names) > 64:
            index = EEG.ch_names.index('Ch64')
            channels_to_drop = []
            try:
                channels_to_drop = EEG.ch_names[index+1:]
            except:
                pass
            EEG.drop_channels(channels_to_drop)

        print(EEG.ch_names)
        matplotlib.use('Agg')  # disable plotting
        raw_plot = EEG.plot(n_channels=len(EEG.ch_names),
                            scalings='auto', show=False)
        raw_file = output_path + subject + '_raw.fif'
        EEG.save(raw_file, overwrite=True)

        # write channel info to the log file
        log.write(str(EEG.info))
        log.write("\n \n \n \n")

        # Rename the channels using the new ch_map

        try:
            EEG.rename_channels(ch_map)
        except:
            print('SOMETHING IS GOING WRONG FOR', subject, mode)
            EEG.rename_channels(ch_map_fix)

        # Now the channels should match the names in the montage
        EEG.set_montage(montage, on_missing='warn')

        # Create a dictionary for channel types
        channel_types = {}

        # Set all channels to 'eeg' by default
        for ch in ch_map.values():
            channel_types[ch] = 'eeg'

        # Update the dictionary with the special channel types
        channel_types['RVEye'] = 'eog'
        channel_types['LHEye'] = 'eog'
        channel_types['RHEye'] = 'eog'
        channel_types['Rneck'] = 'emg'
        channel_types['Lneck'] = 'emg'
        try:
            channel_types['Rmastoid'] = 'misc'
        except:
            pass
        # Retrieve the locations of FP1 and FP2
        fp1_loc = EEG.info['chs'][EEG.ch_names.index('Fp1')]['loc'][:3]
        fp2_loc = EEG.info['chs'][EEG.ch_names.index('Fp2')]['loc'][:3]

        # Compute the average location for FPz
        fpz_loc = (fp1_loc + fp2_loc) / 2

        # Update the location of FPz in the original_EEG object
        EEG.info['chs'][EEG.ch_names.index('GND')]['loc'][:3] = fpz_loc

        # Print the updated location of FPz to verify
        log.write("Updated location of FPz:" + str(fpz_loc) + "\n")

        # Set the channel types in the EEG data
        EEG.set_channel_types(channel_types)

        # Apply a low-pass filter with a cutoff of 50 Hz
        EEG.filter(l_freq=None, h_freq=50)

        # Apply a high-pass filter with a cutoff of 1 Hz
        EEG.filter(l_freq=1, h_freq=None)

        # Add a notch filter from 60 Hz
        # This will create an array [60, 120, 180, 240] to capture the harmonics
        freqs = np.arange(60, 241, 60)
        EEG.notch_filter(freqs)

        # Plot the data to visualize waveforms after filtering
        filtered_plot = EEG.plot(n_channels=len(
            EEG.ch_names), scalings='auto', show=False)

        filtered_plot.set_size_inches(20, 10)
        plt.savefig(output_path+'EEG_filtered.png', dpi=400)

        bad_channel.write('')
        print(EEG.ch_names)
        print(EEG.info)
        # Plotting EEG signal via PSD to check if the notch filter removed the power line noise
        psd_plot = EEG.plot_psd()
        plt.savefig(output_path+'psd.png')
        # Save the filtered data
        # Replace with your desired output directory
        preprocessed_file = output_path + subject + '_maprenamed&nfiltered.fif'
        EEG.save(preprocessed_file, overwrite=True)
        # commit


Reading /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/subjects/116/EC/116.fdt
Reading 0 ... 116599  =      0.000 ...   116.599 secs...


/tmp/ipykernel_3871217/802340082.py:75: RuntimeWarning: Data file name in EEG.data (PCD2018_101_EC_Ch21_replaced.fdt) is incorrect, the file name must have changed on disk, using the correct file name (116.fdt).
  EEG = mne.io.read_raw_eeglab(input_path +'116.set', preload = True)
/tmp/ipykernel_3871217/802340082.py:75: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(input_path +'116.set', preload = True)


Success
['Ch1', 'Ch2', 'Ch3', 'Ch4', 'Ch5', 'Ch6', 'Ch7', 'Ch8', 'Ch9', 'Ch10', 'Ch11', 'Ch12', 'Ch13', 'Ch14', 'Ch15', 'Ch16', 'Ch17', 'Ch18', 'Ch19', 'Ch20', 'Ch22', 'Ch23', 'Ch24', 'Ch25', 'Ch26', 'Ch27', 'Ch28', 'Ch29', 'Ch30', 'Ch31', 'Ch32', 'Ch33', 'Ch34', 'Ch35', 'Ch36', 'Ch37', 'Ch38', 'Ch39', 'Ch40', 'Ch41', 'Ch42', 'Ch43', 'Ch44', 'Ch45', 'Ch46', 'Ch47', 'Ch48', 'Ch49', 'Ch50', 'Ch51', 'Ch52', 'Ch53', 'Ch54', 'Ch55', 'Ch56', 'Ch57', 'Ch58', 'Ch59', 'Ch60', 'Ch61', 'Ch62', 'Ch63', 'Ch64', 'Aux1']
['Ch1', 'Ch2', 'Ch3', 'Ch4', 'Ch5', 'Ch6', 'Ch7', 'Ch8', 'Ch9', 'Ch10', 'Ch11', 'Ch12', 'Ch13', 'Ch14', 'Ch15', 'Ch16', 'Ch17', 'Ch18', 'Ch19', 'Ch20', 'Ch22', 'Ch23', 'Ch24', 'Ch25', 'Ch26', 'Ch27', 'Ch28', 'Ch29', 'Ch30', 'Ch31', 'Ch32', 'Ch33', 'Ch34', 'Ch35', 'Ch36', 'Ch37', 'Ch38', 'Ch39', 'Ch40', 'Ch41', 'Ch42', 'Ch43', 'Ch44', 'Ch45', 'Ch46', 'Ch47', 'Ch48', 'Ch49', 'Ch50', 'Ch51', 'Ch52', 'Ch53', 'Ch54', 'Ch55', 'Ch56', 'Ch57', 'Ch58', 'Ch59', 'Ch60', 'Ch61', 'Ch62', 'Ch63', 

/tmp/ipykernel_3871217/802340082.py:120: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  EEG.set_montage(montage, on_missing='warn')
/tmp/ipykernel_3871217/802340082.py:120: RuntimeWarning: DigMontage is only a subset of info. There is 1 channel position not present in the DigMontage. The channel missing from the montage is:

['Rmastoid'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if this is not an EEG channel, or use the on_missing parameter if the channel position is allowed to be unknown in your analyses.
  EEG.set_montage(montage, on_missing='warn')
/tmp/ipykernel_3871217/802340082.py:153: RuntimeWarning: The unit for channel(s) Rmastoid has changed from V to NA.
  EEG.set_channel_types(channel_types)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



...

/tmp/ipykernel_3871217/802340082.py:120: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  EEG.set_montage(montage, on_missing='warn')
/tmp/ipykernel_3871217/802340082.py:120: RuntimeWarning: DigMontage is only a subset of info. There is 1 channel position not present in the DigMontage. The channel missing from the montage is:

['Rmastoid'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if this is not an EEG channel, or use the on_missing parameter if the channel position is allowed to be unknown in your analyses.
  EEG.set_montage(montage, on_missing='warn')
/tmp/ipykernel_3871217/802340082.py:153: RuntimeWarning: The unit for channel(s) Rmastoid has changed from V to NA.
  EEG.set_channel_types(channel_types)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


['Fp1', 'Fz', 'F3', 'F7', 'LHEye', 'FC5', 'FC1', 'C3', 'T7', 'GND', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'RHEye', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'Lneck', 'PO3', 'POz', 'PO4', 'Rneck', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'RVEye', 'Rmastoid']
<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, LHEye, FC5, FC1, C3, T7, GND, CP5, CP1, Pz, P3, ...
 chs: 58 EEG, 3 EOG, 2 EMG, 1 misc
 custom_ref_applied: False
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 1.0 Hz
 lowpass: 50.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 1000.0 Hz
>
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 2.048 (s)
Overwriting existing file.
Writing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/116/EO/116_maprenamed&nfiltered.f

/tmp/ipykernel_3871217/802340082.py:182: RuntimeWarning: This filename (/home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/116/EO/116_maprenamed&nfiltered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  EEG.save(preprocessed_file, overwrite=True)
